# 导入包

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import readers
import networkx as nx
%matplotlib inline

# 读取数据并构建UI矩阵

In [2]:
avg=0
num=0
def loadData(fp="data_deal/ml-1m/ratings.dat"):
    data = []
    user_set=set()
    item_set=set()
    
    for l in open(fp):
        temp=list(map(int,l.strip().split('::')))
        
        data.append(temp)
        user_set.add(temp[0])
        item_set.add(temp[1])
    data=list(sorted(data,key=lambda x:x[3]))

    return data,user_set,item_set
def dealData(data):
    
    train = []
    le=len(data)
    rating_dict={}
    time_dict={}
    for user, item, rating, timestamp in data:
        addtwodimdict(rating_dict, user, item, rating) # 得分字典 rating_dict
        addtwodimdict(time_dict, user, item, timestamp) # 时间戳字典 time_dict
        global avg,num
        avg+=rating
        num+=1
        train.append((user, item))

    
    # 处理成字典的形式，user->set(items)
    def convert_dict(data):
        data_dict = {}
        for user, item in data:
            if user not in data_dict:
                data_dict[user] = set()
            data_dict[user].add(item)
        data_dict = {k: list(data_dict[k]) for k in data_dict}
        return data_dict

    return convert_dict(train), rating_dict,time_dict
#二维字典的添加
def addtwodimdict(thedict, key_a, key_b, val):
    if key_a in thedict:
        thedict[key_a].update({key_b: val})
    else:
        thedict.update({key_a:{key_b: val}})    

In [3]:
# 获取电影类别字典
def getGenreDict():
    genres=pd.read_csv('best_clusterid.csv')
    genre={}
    for index in genres.index:
        temp=genres.loc[index].values[0]
        genre[index+1]=temp
    return genre

In [4]:
data,user_set,item_set=loadData()

In [5]:
u_i,rating_dict,time_dict=dealData(data)#用户数6040，电影数3952

In [6]:
genre_dict=getGenreDict() # 获取电影类别字典

# 建图

In [7]:
#获取初始用户-电影字典与用户-时间戳字典
def getInitItem(time_dict):
    # 将时间戳字典按时间戳升序
    for user in time_dict:
        sorted(time_dict[user].items(),key=lambda x:x[1],reverse=True)
        
    # 将所有用户的电影序列取出
    user_item_dict={}
    user_time_dict={}
    for user_k,user_v in time_dict.items():
        item_temp=[]
        time_temp=[]
        for item_k,item_v in time_dict[user_k].items(): 
            item_temp.append(item_k)
            time_temp.append(item_v)
        user_item_dict[user_k]=item_temp
        user_time_dict[user_k]=time_temp
        
    return user_item_dict,user_time_dict

In [8]:
# 获取边集
def getEdges(user_item_dict,user_time_dict):
    edges=[]
    users_len=len(user_item_dict)+1
    for user in range(1,users_len):
        items=user_item_dict[user]
        times=user_time_dict[user]
        items_len=len(items)
        for i in range(items_len):
            flag=-1
            for j in range(i+1,items_len):
                if times[i]==times[j]:
                    continue
                if times[i]!=times[j] and flag==-1:
                    temp=(items[i],items[j])
                    edges.append(temp)
                    flag=times[j]
                elif times[j]==flag:
                    temp=(items[i],items[j])
                    edges.append(temp)
                else:
                    break
    return edges

In [9]:
user_item_dict,user_time_dict=getInitItem(time_dict)#获取用户-电影字典与用户-时间戳字典

In [10]:
edges=getEdges(user_item_dict,user_time_dict)#获取边集

In [11]:
# 建图添边
G = nx.DiGraph()
for edge in edges:
    G.add_edge(edge[0], edge[1])

In [12]:
# plt.figure(figsize=(20,20))
# pos = nx.circular_layout(G)

In [13]:
# nx.draw_networkx(G, pos)

# 每个电影相关参数

In [14]:
pagerank_list = nx.pagerank(G, alpha=0.85) # pagerank值
in_d=G.in_degree(G)# 入度
out_d=G.out_degree(G) #出度

In [27]:
# pagerank_list

In [28]:
# in_d

In [29]:
# out_d

# 普通推荐

## 普通推荐

In [15]:
# 获取边集字典(不含重边)
def getEdgeDict(edges):
    set_edges=list(set(edges))# 边集转集合
    set_edges=sorted(set_edges)# 排序
    
    i=1
    temp=[]
    edge_dict={}# 边集字典
    for edge in set_edges:
        if edge[0]==i:
            temp.append(edge)
        else:
            edge_dict.setdefault(i,temp)
            i=i+1
            temp=[]
    edge_dict.setdefault(i,temp)
    return edge_dict

In [16]:
# 获取用户i的所有出点
def getOutNode(i,time_dict,edge_dict):
    # 用户i所有的点
    node_i=set()
    for key in time_dict[i]:
        node_i.add(key)
        
    # 用户i的相关点，包括其本身
    correlation_nodes=set()
    for node in node_i:
        for edge in edge_dict[node]:
            correlation_nodes.add(edge[1])
    
    # 用户i的出点
    out_node=correlation_nodes-node_i
    return out_node

In [17]:
# 获取用户i的PR值
def getNodePR(node,genre_dict,pagerank_list):
    # 替换为pr值
    node=list(node)
    for i in range(len(node)):
        node[i]=(node[i],genre_dict[node[i]],pagerank_list[node[i]])
    return node

In [18]:
# 为用户i推荐10部pr值最大的电影
def recommendTop10(node):
    res=sorted(node,key=lambda x:x[2],reverse=True)#降序排序
    
    ls=[]
    for i in range(0,10):
        ls.append(res[i])
    return ls

In [19]:
edge_dict=getEdgeDict(edges)# 获取边集字典(不含重边)

In [20]:
len(edge_dict)

3952

In [21]:
top10={}
for i in range(1,6040+1):
    out_node=getOutNode(i,time_dict,edge_dict) # 用户i的出点
    PR=getNodePR(out_node,genre_dict,pagerank_list) # 用户i出点的PR值
    ls=recommendTop10(PR) # 选取10个pr值最大的电影
    top10[i]=ls
    break

In [22]:
top10

{1: [(2858, 2857, 0.001789833879293117),
  (858, 857, 0.0014370772244095293),
  (1196, 1195, 0.0014061702908526495),
  (1198, 1197, 0.0013637330042874484),
  (1617, 1616, 0.001274718603129791),
  (589, 588, 0.0012323134838280864),
  (541, 540, 0.0011925837995145304),
  (593, 592, 0.001189498939945671),
  (318, 317, 0.0011563019039531556),
  (110, 109, 0.0011470695734840054)]}

## 新推荐

In [23]:
# 获取边集字典(含重边)
def getRepeatEdgeDict(edges):    
    set_edges=sorted(edges)# 排序
    
    i=1
    temp=[]
    edge_dict={}# 边集字典
    for edge in set_edges:
        if edge[0]==i:
            temp.append(edge)
        else:
            edge_dict.setdefault(i,temp)
            i=i+1
            temp=[]
    edge_dict.setdefault(i,temp)
    return edge_dict

In [24]:
# 获取每个点对每个类的重要程度
def getNodeForGenreImportant(node,repeat_edge_dict,genre_dict):
    n=len(repeat_edge_dict[node])
    if n==0:
        return [0 for i in range(10)]
    
    out_node_genre_count=[0 for i in range(10)]#出点各类别计数
    for edge in repeat_edge_dict[node]:
        out_node=edge[1]#出点
        out_node_genre=genre_dict[out_node]#出点所属类别
        out_node_genre_count[out_node_genre-1] += 1
    
    # 该点对各类的重要程度
    res=[]
    for i in out_node_genre_count:
        res.append(i/n)
    return res    

In [25]:
#获取每个电影对每个类的重要程度的字典
def getImportantDict(repeat_edge_dict,genre_dict):
    res={}
    for i in range(1,3952+1):
        res[i]=getNodeForGenreImportant(i,repeat_edge_dict,genre_dict)
    return res

In [26]:
# 计算每个用户top10的pr值与重要程度的乘积
def getPrMulImportant(top10,important_dict):
    res={}
    for user_k,user_v in top10.items():
        pr=[]
        important=[]
        for item in user_v:
            pr_temp=item[2] # 电影pr值
            important_temp=important_dict[item[0]]#电影的重要程度
            pr.append(pr_temp)
            important.append(important_temp)
        pr=np.mat(pr).reshape(10,1)
        important=np.mat(important)

        res[user_k]=important*pr
    return res

In [27]:
repeat_edge_dict=getRepeatEdgeDict(edges)# 获取边集字典(含重边)
important_dict=getImportantDict(repeat_edge_dict,genre_dict)#获取每个电影对每个类的重要程度的字典
pr_mul_important=getPrMulImportant(top10,important_dict)# 计算每个用户top10的pr值与重要程度的乘积

IndexError: list index out of range

In [ ]:
pr_mul_important

# 迭代推荐

In [44]:
# 添加推荐的边
def addRecommendEdge(nG,new_edges,new_top10):
    for top_k,top_v in new_top10.items():
        #user=time_dict[top_k] # 用户top_k所看的电影-时间戳字典
        item_len=len(time_dict[top_k])# 用户top_k已看电影个数
        items_ls=list(time_dict[top_k].keys()) # 用户top_k已看电影的电影列表
        last_item=items_ls[item_len-1]# 用户top_k所看的最后一个电影的序号
    
        # 添边
        for item in top_v:
            nG.add_edge(last_item,item[0])#在图中添边
            
            temp=(last_item,item[0])
            new_edges.append(temp)#在边集中添边
    return nG,new_edges

In [45]:
#将推荐加入电影-时间戳字典
def addRecommendTimeDict(time_dict,new_top10):
    for top_k,top_v in new_top10.items():
        #user=time_dict[top_k] # 用户top_k所看的电影-时间戳字典
        item_len=len(time_dict[top_k])# 用户top_k已看电影个数
        times_ls=list(time_dict[top_k].values()) # 用户top_k已看电影的时间戳列表
        last_time=times_ls[item_len-1]# 用户top_k所看的最后一个电影的时间戳

        i=1 # 计时器
        for item in top_v:# 推荐的电影
            #user[item[0]]=last_time+i # 推荐的电影加入用户top_k所看的电影时间戳字典
            time_dict[top_k][item[0]]=last_time+i
            i+=1
    return time_dict

In [46]:
# 迭代推荐
def recommend(num,n,nG,new_edges,new_top10,new_time_dict,genre_dict,recommend_res,important_res):
    if num==n+1:
        return recommend_res,important_res
    
    nG,new_edges=addRecommendEdge(nG,new_edges,new_top10) # 添边
    new_time_dict=addRecommendTimeDict(new_time_dict,new_top10) # 获取电影-时间戳字典
    new_pagerank_list = nx.pagerank(nG, alpha=0.85) # pagerank值
    
    #--------------------第一种-----------------------------
    new_edge_dict=getEdgeDict(new_edges)# 获取边集字典(不含重边)

    for i in range(1,6040+1):
        out_node=getOutNode(i,new_time_dict,new_edge_dict) # 用户i的出点
        PR=getNodePR(out_node,genre_dict,pagerank_list) # 用户i出点的PR值
        ls=recommendTop10(PR) # 选取10个pr值最大的电影
        new_top10[i]=ls
        break
    
    recommend_res[num]=new_top10.copy()
    
    #--------------------第二种-----------------------------
    
    new_repeat_edge_dict=getRepeatEdgeDict(new_edges)# 获取边集字典(含重边)
    new_important_dict=getImportantDict(new_repeat_edge_dict,genre_dict)#获取每个电影对每个类的重要程度的字典
    new_pr_mul_important=getPrMulImportant(new_top10,new_important_dict)# 计算每个用户top10的pr值与重要程度的乘积
    
    important_res[num]=new_pr_mul_important.copy()

    #--------------------下一次迭代--------------------------
    recommend_res,important_res=recommend(num+1,n,nG,new_edges,new_top10,new_time_dict,genre_dict,recommend_res,important_res)
    return recommend_res,important_res

In [73]:
# 统计推荐结果中的类
def getCountGenre(recommend_res):
    res={}
    for res_k,res_v in recommend_res.items():#枚举推荐次数
        user={}
        for i in range(1,6040+1):#枚举用户
            ls=[0 for i in range(10)]
            for j in res_v[i]:#枚举top10
                ls[j[1]-1]+=1
            user[i]=ls
            break
        res[res_k]=user
    return res

In [49]:
nG=G.copy()
new_top10=top10.copy()
new_edges=edges.copy()
new_time_dict=time_dict.copy()
recommend_res={}
recommend_res[0]=new_top10.copy()
important_res={}
important_res[0]=pr_mul_important

In [50]:
# 每次迭代的推荐结果和重要程度
#recommand_res[i]/important_res[i]代表第i次迭代的推荐结果/重要程度
#recommand_res[i][j]/important_res[i][j]代表第i次迭代中用户j的推荐结果/重要程度
recommend_res,important_res=recommend(1,2,nG,new_edges,new_top10,new_time_dict,genre_dict,recommend_res,important_res)

In [76]:
# 统计每次迭代推荐结果中的类
# count_genre[i]代表第i次迭代的统计结果
# count_genre[i][j]代表第i次迭代中用户j的统计结果
count_genre=getCountGenre(recommend_res)

{0: {1: [9, 0, 0, 0, 0, 1, 0, 0, 0, 0]},
 1: {1: [4, 1, 0, 2, 0, 3, 0, 0, 0, 0]},
 2: {1: [4, 1, 0, 0, 0, 4, 0, 0, 1, 0]}}

In [77]:
recommend_res

{0: {1: [(2858, 1, 0.001789833879293117),
   (858, 1, 0.0014370772244095293),
   (1196, 1, 0.0014061702908526495),
   (1198, 1, 0.0013637330042874484),
   (1617, 1, 0.001274718603129791),
   (589, 6, 0.0012323134838280864),
   (541, 1, 0.0011925837995145304),
   (593, 1, 0.001189498939945671),
   (318, 1, 0.0011563019039531556),
   (110, 1, 0.0011470695734840054)]},
 1: {1: [(2571, 1, 0.0011324218892936654),
   (1214, 6, 0.0011144564549820596),
   (1221, 1, 0.0011059624823464688),
   (2706, 2, 0.0011003622780941412),
   (2997, 1, 0.001100319033689888),
   (3751, 4, 0.001089572413707119),
   (1210, 6, 0.001052814323857144),
   (50, 1, 0.0010494538794012965),
   (3578, 4, 0.001042210806539038),
   (3481, 6, 0.0010182738938777954)]},
 2: {1: [(3897, 6, 0.0010103489870989192),
   (296, 1, 0.001010251762920457),
   (1127, 2, 0.0010009209762536812),
   (750, 1, 0.0009992391121497384),
   (2396, 6, 0.0009989571978718185),
   (3793, 6, 0.0009975659841270975),
   (912, 1, 0.0009834846814824358)

In [109]:
important_res

{0: {1: matrix([[0.00144479],
          [0.00152788],
          [0.00143036],
          [0.00146568],
          [0.0014931 ],
          [0.00137648],
          [0.00143228],
          [0.00149665],
          [0.00157143],
          [0.00139157]])}, 1: {1: matrix([[0.00088856],
          [0.00088796],
          [0.00089141],
          [0.0008886 ],
          [0.00089091],
          [0.00088796],
          [0.0008886 ],
          [0.00088826],
          [0.00088859],
          [0.00088959]])}, 2: {1: matrix([[0.00083507],
          [0.00083449],
          [0.00084226],
          [0.00083098],
          [0.00083188],
          [0.00083128],
          [0.00083356],
          [0.00083437],
          [0.00083637],
          [0.00083791]])}}

In [ ]:
# 下面的代码都是暂时没用的

# 读取聚类结果与质心

In [16]:
cdata=pd.read_csv("./cdata.csv")

In [17]:
cdata

,0,1,2,3,4,5,6,7,8,9,...,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039
0,4.351003,3.104621,2.706005,2.262216,2.564195,3.551919,2.929935,2.551357,2.133731,3.364310,...,2.717980,2.717682,2.718090,2.718087,2.717717,2.716878,2.717833,2.718275,2.718063,2.717901
1,4.721128,4.310512,4.137632,3.945831,4.172371,4.793374,4.553048,4.265765,3.829143,4.638895,...,4.499089,4.500317,4.499065,4.499225,4.499875,4.501488,4.499479,4.499210,4.498868,4.499742
2,4.321734,3.416504,3.242527,3.022776,3.280313,4.096947,3.661370,3.349515,2.920888,3.788859,...,3.583404,3.583304,3.583433,3.583414,3.583363,3.583213,3.583375,3.583407,3.583471,3.583360
3,4.252725,3.495686,3.394878,3.220026,3.461988,4.228834,3.833575,3.546555,3.109214,3.852128,...,3.779353,3.779516,3.779330,3.779375,3.779519,3.779776,3.779456,3.779329,3.779362,3.779458
4,4.237093,3.517751,3.382041,3.332728,3.487509,4.195591,3.842585,3.672115,3.202708,3.730290,...,3.926770,3.927007,3.926726,3.926753,3.926903,3.927289,3.926899,3.926707,3.926699,3.926839
5,4.033106,2.924828,2.758756,2.502280,2.764834,3.624005,3.148985,2.825437,2.396335,3.308322,...,3.055145,3.054788,3.055277,3.055263,3.054993,3.054291,3.055046,3.055256,3.055358,3.055054
6,4.699018,4.001103,3.825278,3.611371,3.863967,4.673838,4.212145,3.928053,3.508926,4.343539,...,4.168321,4.168886,4.168235,4.168301,4.168728,4.169480,4.168499,4.168262,4.168210,4.168596
7,4.037593,2.987636,2.817626,2.499259,2.799231,3.722445,3.121138,2.787460,2.430204,3.657941,...,3.034181,3.033705,3.034188,3.034231,3.033989,3.033327,3.034096,3.034201,3.034339,3.034065
8,3.928156,3.338338,3.300757,3.451828,3.595491,3.945423,3.910298,3.904663,3.430095,3.774984,...,4.204223,4.204730,4.204080,4.204208,4.204463,4.205228,4.204428,4.204161,4.204031,4.204430
9,4.281143,3.491205,3.371086,3.177759,3.442673,4.125540,3.763937,3.572502,3.106502,3.730170,...,3.860130,3.860351,3.860109,3.860061,3.860223,3.860423,3.860123,3.860103,3.860121,3.860170


# 类间相似度计算 欧氏距离

In [18]:
def eucliDist(A,B):
    A=np.array(A)
    B=np.array(B)
    return np.sqrt(sum(np.power((A-B),2)))

In [19]:
barycenter=[0 for i in range(10)]
for i in range(0,10):
    barycenter[i]=cdata.loc[i]

In [21]:
dist=np.zeros(100).reshape(10,10)
for i in range(0,10):
    for j in range(0,10):
        dist[i][j]=eucliDist(barycenter[i],barycenter[j])

In [22]:
dist

array([[  0.        , 130.23148188,  62.5921162 ,  76.63628354,
         85.318328  ,  23.36906982, 106.786377  ,  22.07314858,
        102.19874497,  81.71471363],
       [130.23148188,   0.        ,  68.06754089,  54.40570699,
         46.96770236, 108.07031138,  23.90054749, 109.13572212,
         33.69225152,  49.63853026],
       [ 62.5921162 ,  68.06754089,   0.        ,  14.24088119,
         23.44654272,  40.15022191,  44.51585524,  41.40954183,
         40.85202799,  19.42039873],
       [ 76.63628354,  54.40570699,  14.24088119,   0.        ,
         10.19355394,  54.04151959,  31.02820601,  55.36442614,
         27.70001198,   6.95963074],
       [ 85.318328  ,  46.96770236,  23.44654272,  10.19355394,
          0.        ,  62.48108181,  24.63593797,  63.88806961,
         18.70286308,   7.63835984],
       [ 23.36906982, 108.07031138,  40.15022191,  54.04151959,
         62.48108181,   0.        ,  84.62426661,   4.53396279,
         79.33399054,  59.12035137],
       [10

In [87]:
genre=10*['']
for i in range(0,10):
    for info in data:
        genre[i]=data.loc[data["genre"]==i]